In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datasets import Dataset, DatasetDict
import warnings
warnings.filterwarnings('ignore')

# Set plotting style and fonts
sns.set_style('whitegrid')
# Increase font sizes for readability
sns.set_context('notebook', font_scale=1.4)
plt.rcParams.update({
    'figure.figsize': (18, 8),
    'font.size': 32,
    'axes.titlesize': 28,
    'axes.labelsize': 25,
    'xtick.labelsize': 23,
    'ytick.labelsize': 23,
    'legend.fontsize': 21
})

In [ ]:
base_dir = Path("../data/evaluation_results")
datasets = ['ft', 'musiccaps', 'lp-musiccaps', 'zero_shot', 'base']

In [ ]:
predictions = {}

for d in datasets:
    print(d)
    model_dir = base_dir / f"{d}_per_sample_scores.csv"
    model_predictions = pd.read_csv(model_dir)
    predictions[d] = pd.DataFrame(model_predictions).head(1000)
display(predictions['ft'].head())

In [ ]:
# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Left subplot: Dataset comparison
datasets_left = ['ft', 'musiccaps', 'lp-musiccaps']
labels_left = ['Fine-tuned VAE Captions', 'MusicCaps Captions', 'LP-MusicCaps Captions']

for m, label in zip(datasets_left, labels_left):
    sns.kdeplot(predictions[m]['clap_score'], label=label, fill=True, alpha=0.5, 
                ax=ax1, linewidth=2.5)

ax1.set_xlabel('CLAP Score', fontweight='bold')
ax1.set_ylabel('Density', fontweight='bold')
# ax1.set_ylim(0.0, 4)
ax1.legend(loc='upper left', framealpha=0.0)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Right subplot: Model comparison
datasets_right = ['ft', 'zero_shot', 'base']
labels_right = ['Fine-tuned VAE Captions', 'Zero-shot Captions', 'Base VAE Captions']

for m, label in zip(datasets_right, labels_right):
    sns.kdeplot(predictions[m]['clap_score'], label=label, fill=True, alpha=0.5, 
                ax=ax2, linewidth=2.5)

ax2.set_xlabel('CLAP Score', fontweight='bold')
ax2.set_ylabel('')
ax2.legend(loc='upper left', framealpha=0.0)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# Overall title
fig.suptitle('CLAP Score Distributions\n(Higher is Better)', 
             fontweight='bold', y=1.00)

plt.style.use('petroff10')
plt.tight_layout()
plt.savefig("../docs/assets/clap_score_distribution_comparison.pdf", bbox_inches='tight')
plt.show()

In [ ]:
data = {
    'lp-musiccaps': 0.39584,
    'musiccaps': 0.42406,
    'base': 0.3906,
    'zero_shot': 0.5048,
    'ft': 0.4241
}

fig, (ax1, ax2) = plt.subplots(1, 2)

# Left subplot: Dataset comparison
datasets_left = ['ft', 'musiccaps', 'lp-musiccaps']
labels_left = ['ConceptCaps', 'MusicCaps', 'LP-MusicCaps']

for m, label in zip(datasets_left, labels_left):
    ax1.bar(label, data[m], alpha=0.7)
ax1.set_xlabel('Dataset', fontweight='bold')
ax1.set_ylabel('FAD Score', fontweight='bold')
ax1.set_ylim(0.0, 0.6)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Right subplot: Model comparison
datasets_right = ['ft', 'zero_shot', 'base']
labels_right = ['Fine-tuned LLM', 'Zero-shot', 'Base LLM']

for m, label in zip(datasets_right, labels_right):
    ax2.bar(label, data[m], alpha=0.7)
ax2.set_xlabel('Model', fontweight='bold')
ax2.set_ylim(0.0, 0.6)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# Overall title
fig.suptitle('FAD Score Comparison\n(Lower is Better)', 
             fontweight='bold', y=1.00)

plt.style.use('petroff10')
plt.tight_layout()
plt.savefig("../docs/assets/fad_score_comparison.pdf", bbox_inches='tight')
plt.show()